# INFLATION AND FERTILITY (MANGLER TEKST TIL FERTILITET)
### Motivation
This dataproject show how fertility and inflation have changed over time, understanding this dynamic can be crucial to understand the underlying trends and policies that have affected the economy over time.

Inflation rates, can be interpreted as the health of the economy wheras a high inflation can indicate economic instability, affecting everything from consumer purchasing power to investments in the economy. Conversely, low inflation present the economic predictability and the potential growth rate. By tracking inflation trends, policymakers can assess the effectiveness of the monetary policy and adjust it like wise.

Fertility rates, which measure the  number of children born per woman, offer essential insights into demographic shifts and social impact. Changes in fertility rates can potential influence population growth, age distribution, and workforce dynamics. Where low fertility rates,  can lead to an aging population, affecting pension systems and the social system. We assume that policymakers rely on fertility data to plan the needs in the economy.

In conclusion, by examining inflation and fertility trends over time, we gain a  understanding of economic resilience, demographic shifts, and the interconnectedness of economic and social policies. Whoch can be used by a policy maker to enhance policy which raises the growth rates and ensures a sustainaible welfare of the economy.


We have used a rather long period (1995-2023) so we can perform OLS estimation on our dataset but we have still shown our ability sort in the years selected


### The setup
For this project we use Statistiskbanken to get a comprehensive overview of the avaliable data of the danish economy. where we look at: CPI, GDP and Fertility Rate
- The Consumer Price Indes is used as a proxy for the inflation and represents the average change over time in prices paid by consumers for a variety of goods and services, reflecting inflationary pressures in the economy. 

- The GDP represents the total monetary value of all goods and services produced within Denmarks borders over a specific period, serving as a measure of economic output and growth. 

- The fertility rate represents the average number of children born per woman, providing insights into population growth and demographic trends.

Not this study does not look at the causality, due to the fact we do apply regression analysis and hypothesis testing and is a descriptive study.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Notes: 

We do not use the dataproject.py as we feel the datasorting and merging is essential to understand this dataproject and displays our thought process in this assignment.

For the "FOD33" we have retrieved the data through excel and inserted into our dataproject we do this, hence the Api only registers "TID" and "ALDER" as variables but not the fertility, this variable is essential thus it is retrieved "manually".

furthermore for the Regressions we have used precompiled lazy dataset otherwise it could not read it. 


Imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import statsmodels.api as sm
from matplotlib_venn import venn2
import pandas_datareader # install with `pip install pandas-datareader`
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

import plotly.express as px
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# for the excel data
import os 

# Using assert to check that paths exist on computer.
assert os.path.isdir('data/')
assert os.path.isfile('data/bog1.xlsx')


# Print everything in data
os.listdir('data/')

# Read and clean data

Import your data, either through an API or manually, and load it. 

In [ ]:
fert = DstApi('FERT1')
gdp = DstApi('NAN1')
fpi = DstApi ('PRIS112')

We have found our desired variables now it is time to clean the data

In [ ]:
tabsum_fert= fert.tablesummary(language='en')
display(tabsum_fert)


In [ ]:
# The available values for a each variable: 
for variable in tabsum_fert['variable name']:
    print(variable+':')
    display(fert.variable_levels(variable, language='en'))

In [ ]:
tabsum_gdp= gdp.tablesummary(language='en')
display(tabsum_gdp)
for var in tabsum_gdp['variable name']:
    print(var+':')
    display(gdp.variable_levels(var, language='en'))


In [ ]:
tabsum_fpi= fpi.tablesummary(language='en')
display(tabsum_fpi)
for var in tabsum_fpi['variable name']:
    print(var+':')
    display(fpi.variable_levels(var, language='en'))


Now we have a quick overview of the avaible datapoints

In [ ]:
params = fert.define_base_params(language='en')
params

In [ ]:
params ={'table': 'fert1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'HERKOMST', 'values': ['AK']},
  {'code': 'Tid', 'values': ['>1990<=2023']}]}

In [ ]:
fert_api = fert.get_data(params=params)
fert_api.head(15)

In [ ]:
params = gdp.define_base_params(language='en')
params

In [ ]:
params ={'table': 'nan1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'TRANSAKT', 'values': ['B1GQK']},
  {'code': 'PRISENHED', 'values': ['V_M']},
  {'code': 'Tid', 'values': ['>1990<=2023']}]}

In [ ]:
gdp_api = gdp.get_data(params=params)
gdp_api.head(15)

In [ ]:
params = fpi.define_base_params(language='en')
params

In [ ]:
params = {'table': 'pris112',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'HOVED', 'values': ['1005']},
  {'code': 'Tid', 'values': ['>1990<=2023']}]}

In [ ]:
fpi_api = fpi.get_data(params=params)
fpi_api.head(15)

We have now found the desired data where we want to merge it into one dataset.

The excel datasheet is extracted:

In [ ]:
filename = 'data/ferrrr.xlsx' # open the file and have a look at it
pd.read_excel(filename).head(10)
fertreg = pd.read_excel(filename, skiprows=0)
fertreg.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)

In [ ]:
filename = 'data/BOG1.xlsx' # open the file and have a look at it
pd.read_excel(filename).head(10)

In [ ]:
fertAlt = pd.read_excel(filename, skiprows=2)
fertAlt.drop(columns=[str(i) for i in range(1973, 1994)], inplace=True)
fertAlt.rename(columns = {'Unnamed: 0':'Age group'}, inplace=True)

In [ ]:
col_dict = {}
for i in range(1994, 2023+1): # range goes from 2010 to 2023
    col_dict[str(i)] = f'fertAlt{i}'

col_dict = {str(i) : f'fertAlt{i}' for i in range(1991,2023+1)}
col_dict

In [ ]:
fertAlt.rename(columns = col_dict, inplace=True)
fertAlt.head(7)

Here we apply data cleaning so it can read the data.

## Merging

We want to merge our three data sets into one to draw a graph.

In [ ]:
# Merge fpi_api and fert_api on 'TID'
merged_first = pd.merge(fpi_api, fert_api, on='TID', how='inner')


In [ ]:
# Now merge the result with gdp_api on 'TID'
final_merged = pd.merge(merged_first, gdp_api, on='TID', how='inner')


In [ ]:
merged_first.rename(columns={
    'INDHOLD_x': 'FertilityRate',
    'INDHOLD_y': 'ConsumerPriceIndex'
}, inplace=True)

# Assuming gdp_api has been loaded and contains a column that should be renamed to 'GrossDomesticProduct'
gdp_api.rename(columns={
    'INDHOLD': 'GrossDomesticProduct'
}, inplace=True)

# Merge the result with gdp_api on 'TID'
final_merged = pd.merge(merged_first, gdp_api, on='TID', how='inner')

# Select only the desired columns
final_selected = final_merged[['TID', 'FertilityRate', 'ConsumerPriceIndex', 'GrossDomesticProduct']]

print(final_selected)



In [ ]:
# Convert columns to numeric, handling non-convertible values
final_selected.loc[:, 'FertilityRate'] = pd.to_numeric(final_selected['FertilityRate'], errors='coerce')
final_selected.loc[:, 'ConsumerPriceIndex'] = pd.to_numeric(final_selected['ConsumerPriceIndex'], errors='coerce')
final_selected.loc[:, 'GrossDomesticProduct'] = pd.to_numeric(final_selected['GrossDomesticProduct'], errors='coerce')

# Calculate year-over-year percentage changes
final_selected.loc[:, 'FertilityRate_Pct_Change'] = final_selected['FertilityRate'].pct_change() * 100
final_selected.loc[:, 'ConsumerPriceIndex_Pct_Change'] = final_selected['ConsumerPriceIndex'].pct_change() * 100
final_selected.loc[:, 'GrossDomesticProduct_Pct_Change'] = final_selected['GrossDomesticProduct'].pct_change() * 100


## Drawing the Graphs

We draw histograms 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

fertAlt[['Age group', 'fertAlt1995']].plot(x='Age group', y='fertAlt1995', kind='bar', ax=axes[0, 0])
axes[0, 0].set_xlabel('Age group')
axes[0, 0].set_ylabel('Fertility rate')
axes[0, 0].set_title('Fertility Rate by Age Group (1995)')

fertAlt[['Age group', 'fertAlt2005']].plot(x='Age group', y='fertAlt2005', kind='bar', ax=axes[0, 1])
axes[0, 1].set_xlabel('Age group')
axes[0, 1].set_ylabel('Fertility rate')
axes[0, 1].set_title('Fertility Rate by Age Group (2005)')

fertAlt[['Age group', 'fertAlt2015']].plot(x='Age group', y='fertAlt2015', kind='bar', ax=axes[1, 0])
axes[1, 0].set_xlabel('Age group')
axes[1, 0].set_ylabel('Fertility rate')
axes[1, 0].set_title('Fertility Rate by Age Group (2015)')

fertAlt[['Age group', 'fertAlt2020']].plot(x='Age group', y='fertAlt2020', kind='bar', ax=axes[1, 1])
axes[1, 1].set_xlabel('Age group')
axes[1, 1].set_ylabel('Fertility rate')
axes[1, 1].set_title('Fertility Rate by Age Group (2015)')

plt.tight_layout()
plt.show()

We print the actual values in a table and more importantly draw the graph

In [ ]:
pct_change_df = final_selected[['TID', 'FertilityRate_Pct_Change', 'ConsumerPriceIndex_Pct_Change', 'GrossDomesticProduct_Pct_Change']]
pct_change_df = pct_change_df.iloc[1:].reset_index(drop=True)


print("Year-Over-Year Percentage Changes (Preview):")
print(pct_change_df.head(40).to_string(index=False))

In [ ]:
# Create an interactive line plot with Plotly
fig = px.line(final_selected, x='TID', y=[
    'FertilityRate_Pct_Change', 'ConsumerPriceIndex_Pct_Change', 'GrossDomesticProduct_Pct_Change'],
    labels={
        'value': 'Percentage Change (%)',
        'variable': 'Metrics',
        'TID': 'Year'
    },
    title='Year-over-Year Percentage Change'
)

# Update layout for clearer display
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Percentage Change',
    legend_title='Metric'
)

# Show the plot
fig.show()




If our interactive graph cant load, we have also created a simple sketch using matplot package, by removing the "#" but we advise to use the interactive figure, as you can isolate each metric 

In [ ]:
#plt.figure(figsize=(10, 6))
#plt.plot(final_selected['TID'], final_selected['FertilityRate_Pct_Change'], label='Fertility Rate % Change', marker='o')
#plt.plot(final_selected['TID'], final_selected['ConsumerPriceIndex_Pct_Change'], label='Consumer Price Index % Change', marker='x')
#plt.plot(final_selected['TID'], final_selected['GrossDomesticProduct_Pct_Change'], label='Gross Domestic Product % Change', marker='^')
#plt.xlabel('Year')
#plt.ylabel('Percentage Change (%)')
#plt.title('Year-over-Year Percentage Change')
#plt.legend()
#plt.grid(True)

#plt.show()

# Ordinary Least Squares

Finally we run a number of OLS on the fertility rate and the CPI and the GDP to see how they affect eachother

In [ ]:
### 1
# Define the dependent variable 'y' and independent variables 'X'
y = fertreg['FERTILITY']
X = fertreg[['GDP', 'CPI']]

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)


# Fit the model
model1 = sm.OLS(y, X).fit()


# Print out the statistics
print(model1.summary())


In [ ]:
### 2

# Define the dependent variable 'y' and independent variables 'X'
y = fertreg['15-19']
X = fertreg[['GDP', 'CPI']]

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the model
model2 = sm.OLS(y, X).fit()

# Print out the statistics
print(model2.summary())

In [ ]:
### 3
# Define the dependent variable 'y' and independent variables 'X'
y = fertreg['30-34']
X = fertreg[['GDP', 'CPI']]

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the model
model3 = sm.OLS(y, X).fit()

# Print out the statistics
print(model3.summary())

## Analysis:

For this Data-project we have conducted a simple analysis of three variables from Statiskbanken, we have utilised a common tool from the descriptive economcis course where we look at the yearly percentage change in a variable, thus being able to compare the data to see how the varaibles changes within the analytic period.


#### What does the box diagram show?

We observe that is a shift in who has the highest fertility in the four samples, we observe, that in 1995, it is the age group 25-29 who has the highest fertility and in the rest of the sample we find that the age group 30-34 has the highest fertility, where the age group 25-29 consistently is declining. Almost in all periods we observe that the age group 15-19 and 45-49 has the lowest fertility on almost around 0.0, thus we find it intersting to see if we can run a number of simple regression and observe if our two variables can affect these changes.

#### What does the graph show?

The Gross Domestic Product reflects the economic output and growth over time.
- The Gross Domestic Product starts with an yearly increase on 5 pct. points. after adjusting to a 2.0 pct. points. in the period 2001-2003. from 2004 to 2008 we see a yearly increase on around 5.0 to 3.0 pct. points. where as in 2009 we see decrease on -4.4 pct. points. this is caused by the financial crisis. followed the next year 2010 with an increase on 4.8 pct points. Onwards we see an increase on around 2.0 pct. points. increase each year with the exception of 2016-2017 followed by a sharp decling in 2020 where it only increases with 0,5 pct. following by a boom in 2021-2022 where it increases with 9-11 pct.points. - Overall this draws a picture of a stable economy recovering from the financial crisis with a boom in 2021-2022.

The Consumer Price Index tracks changes in the cost of living and inflationary pressures.
- The Consumer Price Index change is more volatile. This is evident in the period 1995-1996 where it starts off with -3.1 pct. points. and bounces back on an increase on 0.15 pct. points. this volatiltiy describes the whole period up to 2010 where it bounces between a negative and positve value. It experiences a significant drop around 2011, a rise peaking in 2012, and then falls again sharply by 2023. The most notable point is the extreme dip in 2022, indicating a period of deflation or significant price decreases, followed by a recovery in 2023.
- Overall we can see that the covid crisis had a massive effect on the consumer prices during the crisis, while the financial crisis had effects after the crisis.

The Fertility Rate provides insights into demographic trends and societal responses.
- Fertility seems stable compared to the others, with fluctuations remaining within a narrower range. but with a steady decline wheras the first period from 1995-2003 we see an yearly increaseover 2.0 pct. points. whereas in 2004-2009 we only see an increase by 1 pct. point. with the exception of 2008 we see a fertility increase by 3.0 pct. points. in 2010-2013 we can see an increase by 2.0-3.0 pct. points. We observe around the years 2014-2015, after which it stabilizes and shows an upward trend starting around 2020, peaking in 2022. butexcluding the covid years it fluctuates between 3-1 yearly pct. change. 


- The stabilazation can be seen as a indicator stable policy changes and economic confidence, which can be interpreted as a society which trusts the government during crises while due to “post-crisis fiscal stimuli packages” or other growth-driving factors.

#### What does the OLS show?

We run a number of regressions to see how the age groups respond to 

We have set up a model which sees how fertility is affected by the GDP, and the Consumer Price Index. We find that the fertility is affected negativly by the Consumer Price index, the coefficient is -0.2389, indicating that for each 1% increase in the CPI percentage change, the fertility rate percentage change is expected to decrease by approximately 0.24%, holding GDP constant.

For the GDP the The coefficient is 0.2453, meaning that for each 1% increase in GDP percentage change, the fertility rate percentage change is expected to increase by approximately 0.25%, holding CPI constant

The p-values for both independent variables are 0.000, which indicates that these variables are statistically significant predictors of the fertility rate percentage.


## Conclusion on the graph: 
The data project of the  GDP, CPI, and Fertility Rate in Denmark reveals  relationships and dynamic interactions within the economy where the key take aways are

- __Economic adaptation__ The danish GDP show effective ecnomic adaptation in recovery phases following economic downturns, which can be interpretated as robust economic management by the policy makers.

- __Price stability__ The fluctutations of the Consumer Price Index shows how policy changes and externals shocks in the economy affects the sensitvity of the consumer prices

The result we wanted to find was that the relationship between CPI and Fertility would affect the cost of living. Where an increase in  Consumer Price Index, would indicate inflation, which raises the cost of living.  We know that If wages do not keep up with inflation, families may feel financially pressured and may decide to have fewer children. This didn’t happen due to the overall stable nature of the Danish Economy.


## Conclusion on the OLS

This OLS regression model suggests that there is a statistically significant relationship between the percentage changes in the Consumer Price Index and GDP with the percentage change in the fertility rate. With CPI having a negative impact and GDP having a positive impact on fertility rates. The model appears to be statistically valid with normally distributed residuals and significant predictors.

Which means that higher inflation or cost of living is associated with a decline in fertility rates. This suggests that when the cost of goods and services rises, people may feel financial pressure, leading them to have fewer children. Reversly Economic growth is associated with an increase in fertility rates seen through the GDP. This implies that when the economy is doing well, people may feel more financially secure and confident in their ability to support a larger family, leading to more children being born.

